In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan' at line 17)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan' at line 17)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS    
         lp__  927.984290500  3.0451569045 0.048148158253 0.0998707692  929.699421
accept_stat__    0.940335511  0.0831214800 0.001314265997 0.0014697124 1000.000000
   stepsize__    0.053168800  0.0019641763 0.000031056354 0.0003144806   39.009752
  treedepth__    5.818000000  0.4630654461 0.007321707578 0.0113069074 1000.000000
 n_leapfrog__   68.450000000 29.0868528936 0.459903525550 0.6649137210 1000.000000
  divergent__    0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__ -921.484384500  3.9624337964 0.062651579371 0.1239145894 1000.000000
            a    1.097854504  0.7026094681 0.011109231124 0.0235815521  887.734752
           bp    0.260969592  0.0758161757 0.001198758994 0.0025237497  902.466713
  a_society.1   -0.197211604  0.2373795407 0.003753300

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*